In [10]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import numpy as np
import os

os.environ['WDM_LOG'] = '0'

In [ ]:


def scrape_metrocuadrado(tipo_negocio, ciudad, max_pages=3):
    """
    Scrapea listados de venta o arriendo en Metrocuadrado.com
    tipo_negocio: "venta" o "arriendo"
    ciudad: e.g. "medellin"
    max_pages: número máximo de páginas a recorrer
    """
    base_url = f"https://www.metrocuadrado.com/apartamentos-casas/{tipo_negocio}/{ciudad}/"

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--log-level=3")
    options.add_experimental_option("excludeSwitches", ["enable-logging"])

    driver = webdriver.Chrome(options=options)
    listings = []

    for page in range(1, max_pages + 1):
        url = base_url if page == 1 else f"{base_url}?pagina={page}"
        print(f"Scrapeando página {page}: {url}")
        driver.get(url)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.property-card__container"))
            )
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.select("div.property-card__container")

            for card in cards:
                try:
                    titulo = card.select_one("div.property-card__detail-title h2")
                    titulo = titulo.get_text(strip=True) if titulo else None

                    precio = card.select_one("div.property-card__detail-price")
                    precio = precio.get_text(strip=True) if precio else None

                    ubicacion = card.select_one("div.property-card__detail-top__left div")
                    ubicacion = ubicacion.get_text(strip=True) if ubicacion else None

                    specs = card.select_one("pt-main-specs")
                    area = specs.get("squaremeter") if specs else None
                    banos = specs.get("toilets") if specs else None
                    habitaciones = specs.get("bedrooms") if specs else None
                    parqueaderos = specs.get("parking") if specs else None

                    link = card.select_one("a[href]")
                    link = "https://www.metrocuadrado.com" + link["href"] if link else None

                    listings.append({
                        "titulo": titulo,
                        "precio": precio,
                        "ubicacion": ubicacion,
                        "area_m2": area,
                        "banos": banos,
                        "habitaciones": habitaciones,
                        "parqueaderos": parqueaderos,
                        "link": link
                    })
                except Exception as e:
                    print("Error en tarjeta:", e)

        except Exception as e:
            print(f"Error en página {page}:", e)
            break

        time.sleep(2)  # Pausa para no sobrecargar el sitio

    driver.quit()
    return pd.DataFrame(listings)


if __name__ == "__main__":
    venta_df = scrape_metrocuadrado("venta", "medellin", max_pages=200)
    arriendo_df = scrape_metrocuadrado("arriendo", "medellin", max_pages=200)

    venta_df.to_csv("metrocuadrado_venta_medellin.csv", index=False, encoding="utf-8-sig")
    arriendo_df.to_csv("metrocuadrado_arriendo_medellin.csv", index=False, encoding="utf-8-sig")

    print("Scraping completado ✅. Archivos guardados.")


Scrapeando página 1: https://www.metrocuadrado.com/apartamentos-casas/venta/medellin/


KeyboardInterrupt: 

In [2]:
dfa=pd.read_csv("metrocuadrado_venta_medellin.csv")
dfb=pd.read_csv("metrocuadrado_arriendo_medellin.csv")

print(dfa.head())

                                              titulo          precio  \
0                 Casa en Venta, El Tesoro, Medellín  $1.850.000.000   
1         Apartamento en Venta, San German, Medellín    $590.000.000   
2  GUAYACANES - Apartamento en Venta, CIUDAD DEL ...    $950.000.000   
3  River Park - Apartamento en Venta, CIUDAD DEL ...    $701.625.000   
4  SAMÁN  - Apartamento en Venta, CIUDAD DEL RIO,...    $640.000.000   

                   ubicacion  area_m2  banos  habitaciones  parqueaderos  \
0       El Tesoro | Medellín    293.0      4             3             2   
1      San German | Medellín     74.0      2             3             1   
2  CIUDAD DEL RIO | Medellín     96.0      3             3             2   
3  CIUDAD DEL RÍO | Medellín     57.5      2             2             1   
4  CIUDAD DEL RIO | Medellín     56.0      2             1             1   

                                                link  
0  https://www.metrocuadrado.com/inmueble/venta-c...  


In [3]:
dfc=dfa.copy()
dfd = dfb.copy()

In [4]:
dfa['tipo'] = dfa['titulo'].apply(lambda x: 'Apartamento' if 'apartamento' in x.lower() else ('Casa' if 'casa' in x.lower() else 'Otro'))
dfb['tipo'] = dfb['titulo'].apply(lambda x: 'Apartamento' if 'apartamento' in x.lower() else ('Casa' if 'casa' in x.lower() else 'Otro'))
dfa['titulo'] = dfa['titulo'].str.replace(r'\bMedellín\b\s*', '', case=False, regex=True)
dfb['titulo'] = dfb['titulo'].str.replace(r'\bMedellín\b\s*', '', case=False, regex=True)
dfa['ubicacion'] = dfa['ubicacion'].str.replace(r'\bMedellín\b\s*', '', case=False, regex=True).str.replace('|','')
dfb['ubicacion'] = dfb['ubicacion'].str.replace(r'\bMedellín\b\s*', '', case=False, regex=True)
dfa

,titulo,precio,ubicacion,area_m2,banos,habitaciones,parqueaderos,link,tipo
0,"Casa en Venta, El Tesoro,",$1.850.000.000,El Tesoro,293.0,4,3,2,https://www.metrocuadrado.com/inmueble/venta-c...,Casa
1,"Apartamento en Venta, San German,",$590.000.000,San German,74.0,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
2,"GUAYACANES - Apartamento en Venta, CIUDAD DEL ...",$950.000.000,CIUDAD DEL RIO,96.0,3,3,2,https://www.metrocuadrado.com/proyecto/guayaca...,Apartamento
3,"River Park - Apartamento en Venta, CIUDAD DEL ...",$701.625.000,CIUDAD DEL RÍO,57.5,2,2,1,https://www.metrocuadrado.com/proyecto/river-p...,Apartamento
4,"SAMÁN - Apartamento en Venta, CIUDAD DEL RIO,",$640.000.000,CIUDAD DEL RIO,56.0,2,1,1,https://www.metrocuadrado.com/proyecto/saman/1...,Apartamento
...,...,...,...,...,...,...,...,...,...
1675,"SAMÁN - Apartamento en Venta, CIUDAD DEL RIO,",$850.000.000,CIUDAD DEL RIO,80.0,3,2,1,https://www.metrocuadrado.com/proyecto/saman/1...,Apartamento
1676,"SAMÁN - Apartamento en Venta, CIUDAD DEL RIO,",$640.000.000,CIUDAD DEL RIO,56.0,2,1,1,https://www.metrocuadrado.com/proyecto/saman/1...,Apartamento
1677,"GUAYACANES - Apartamento en Venta, CIUDAD DEL ...",$950.000.000,CIUDAD DEL RIO,96.0,3,3,2,https://www.metrocuadrado.com/proyecto/guayaca...,Apartamento
1678,"Apartamento en Venta, Castropol,",$880.000.000,Castropol,86.0,3,3,2,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento


In [5]:
dfa['precio']= dfa['precio'].str.replace(r'[^\d]', '', regex=True).astype(float)
dfb['precio']= dfb['precio'].str.replace(r'[^\d]', '', regex=True).astype(float)
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   titulo        1680 non-null   object 
 1   precio        1680 non-null   float64
 2   ubicacion     1680 non-null   object 
 3   area_m2       1680 non-null   float64
 4   banos         1680 non-null   int64  
 5   habitaciones  1680 non-null   int64  
 6   parqueaderos  1680 non-null   int64  
 7   link          1680 non-null   object 
 8   tipo          1680 non-null   object 
dtypes: float64(2), int64(3), object(4)
memory usage: 118.3+ KB


In [6]:
dfa['ubicacion']=dfa['ubicacion'].str.lower().str.strip()
dfa[dfa['ubicacion'] =='belen la palma']

,titulo,precio,ubicacion,area_m2,banos,habitaciones,parqueaderos,link,tipo
87,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
229,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
366,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
802,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
912,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
946,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
1119,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
1478,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento
1593,"Apartamento en Venta, Belen La Palma,",880000000.0,belen la palma,95.06,2,3,1,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento


In [7]:
dfb['ubicacion']=dfb['ubicacion'].str.lower().str.strip()
dfb#[dfb['ubicacion'] =='belen la palma']

,titulo,precio,ubicacion,area_m2,banos,habitaciones,parqueaderos,link,tipo
0,"Apartamento en Arriendo, Milla De Oro,",7000000.0,milla de oro |,185,4,3,2,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
1,"Apartamento en Arriendo, LA AMERICA,",2300000.0,la america | noroccidente |,61,2,2,1,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
2,"Apartamento en Arriendo, Laureles,",3300000.0,laureles | noroccidente |,120,2,3,1,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
3,"Apartamento en Arriendo, Poblado,",6500000.0,poblado | suroriente |,130,3,3,1,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
4,"Casa en Arriendo, Poblado,",14000000.0,poblado | suroriente |,256,5,4,2,https://www.metrocuadrado.com/inmueble/arriend...,Casa
...,...,...,...,...,...,...,...,...,...
1666,"Casa en Arriendo, Poblado,",14000000.0,poblado | suroriente |,256,5,4,2,https://www.metrocuadrado.com/inmueble/arriend...,Casa
1667,"Apartamento en Arriendo, Poblado,",5500000.0,poblado | suroriente |,120,2,3,2,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
1668,"Apartamento en Arriendo, ESTADIO,",4000000.0,estadio | noroccidente |,140,3,2,2,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
1669,"Apartamento en Arriendo, Laureles,",2700000.0,laureles | noroccidente |,100,2,3,1,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento


In [12]:
np.savez_compressed('datos.npz', arr=dfb, ven=dfa)


In [25]:
with np.load('datos.npz', allow_pickle=True) as data:
    arreindo_data = data['arr']
    venta_data = data['ven']
arreindo_data

array([['Apartamento en Arriendo, Milla De Oro, ', 7000000.0,
        'milla de oro |', ..., 2,
        'https://www.metrocuadrado.com/inmueble/arriendo-apartamento-medellin-milla-de-oro-3-habitaciones-4-banos-2-garajes/5606-M5974739',
        'Apartamento'],
       ['Apartamento en Arriendo, LA AMERICA, ', 2300000.0,
        'la america | noroccidente |', ..., 1,
        'https://www.metrocuadrado.com/inmueble/arriendo-apartamento-medellin-la-america-2-habitaciones-2-banos-1-garajes/20692-M5982689',
        'Apartamento'],
       ['Apartamento en Arriendo, Laureles, ', 3300000.0,
        'laureles | noroccidente |', ..., 1,
        'https://www.metrocuadrado.com/inmueble/arriendo-apartamento-medellin-lorena-3-habitaciones-2-banos-1-garajes/2511-M4911512',
        'Apartamento'],
       ...,
       ['Apartamento en Arriendo, ESTADIO, ', 4000000.0,
        'estadio | noroccidente |', ..., 2,
        'https://www.metrocuadrado.com/inmueble/arriendo-apartamento-medellin-estadio-2-habitaci

In [27]:
arriendo = pd.DataFrame(arreindo_data, columns=dfb.columns)
venta = pd.DataFrame(venta_data, columns=dfa.columns)
arriendo


,titulo,precio,ubicacion,area_m2,banos,habitaciones,parqueaderos,link,tipo
0,"Apartamento en Arriendo, Milla De Oro,",7000000.0,milla de oro |,185,4,3,2,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
1,"Apartamento en Arriendo, LA AMERICA,",2300000.0,la america | noroccidente |,61,2,2,1,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
2,"Apartamento en Arriendo, Laureles,",3300000.0,laureles | noroccidente |,120,2,3,1,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
3,"Apartamento en Arriendo, Poblado,",6500000.0,poblado | suroriente |,130,3,3,1,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
4,"Casa en Arriendo, Poblado,",14000000.0,poblado | suroriente |,256,5,4,2,https://www.metrocuadrado.com/inmueble/arriend...,Casa
...,...,...,...,...,...,...,...,...,...
1666,"Casa en Arriendo, Poblado,",14000000.0,poblado | suroriente |,256,5,4,2,https://www.metrocuadrado.com/inmueble/arriend...,Casa
1667,"Apartamento en Arriendo, Poblado,",5500000.0,poblado | suroriente |,120,2,3,2,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
1668,"Apartamento en Arriendo, ESTADIO,",4000000.0,estadio | noroccidente |,140,3,2,2,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
1669,"Apartamento en Arriendo, Laureles,",2700000.0,laureles | noroccidente |,100,2,3,1,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento
